In [2]:
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [37]:
df = pd.read_csv("./res/insurance_data.csv")

In [38]:

df.sample(5)

,age,affordibility,bought_insurance
12,27,0,0
18,19,0,0
11,28,1,0
17,58,1,1
3,52,0,0


In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']],df.bought_insurance,test_size=0.2)

In [40]:
X_train['age'] = X_train['age']/100
X_test['age'] = X_test['age'] / 100

In [46]:
model = keras.Sequential([
    keras.Input(shape=(2,)),
    keras.layers.Dense(1,activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train,y_train, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step - accuracy: 0.5000 - loss: 0.7155
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 0.7152
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.7148
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5000 - loss: 0.7144
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.7140
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.7136
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.7133
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5000 - loss: 0.7129
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5000 - loss: 0.7125
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5000 - loss: 0.7121
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5000 - loss: 0.7118
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy

In [47]:
model.evaluate(X_test,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 1.0000 - loss: 0.2973


[0.2972545623779297, 1.0]

In [48]:
model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.3688309 ],
       [0.7883222 ],
       [0.194536  ],
       [0.80413157],
       [0.82614326],
       [0.3688309 ]], dtype=float32)

In [49]:
y_test

10    0
5     1
13    0
17    1
9     1
19    0
Name: bought_insurance, dtype: int64

In [50]:
coef,intercept = model.get_weights()
coef, intercept

(array([[4.873889 ],
        [1.4196932]], dtype=float32),
 array([-2.834229], dtype=float32))

In [79]:
class MyNN:
    def __init__(self) -> None:
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def get_weights(self):
        return self.w1, self.w2, self.bias
    
    def predict(self,X):
        weighted_sum = X['age']*self.w1 + self.w2*X['affordibility'] + self.bias
        return self._sigmoid_numpy(weighted_sum)

    def fit(self, X_train, y_train, epochs):
        self._gradient_descent(age=X_train['age'], affordibility=X_train['affordibility'],y_true=y_train,epochs=epochs,threshold=0.4847)

    def _gradient_descent(self, age, affordibility, y_true, epochs, threshold):
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = self.w1*age+self.w2*affordibility+self.bias
            y_predicted = self._sigmoid_numpy(weighted_sum)
            loss = self._log_los(y_true,y_predicted)
            w1d = (1/n)*np.dot(np.transpose(age), y_predicted-y_true)
            w2d = (1/n)*np.dot(np.transpose(affordibility), y_predicted-y_true)
            bias_n = np.mean(y_predicted-y_true)

            self.w1 = self.w1 - rate*w1d
            self.w2 = self.w2 - rate*w2d
            self.bias = self.bias- rate*bias_n
            if loss <= threshold:
                print(f"Epoch:{i} w1:{self.w1}, w2:{self.w2} bias:{self.bias} Loss:{loss}")
                break
            print(f"Epoch:{i} w1:{self.w1}, w2:{self.w2} bias:{self.bias} Loss:{loss}")

        return self.w1, self.w2, self.bias


    def _log_los(self, y_true, y_predicted):
        e = 1e-15
        y_predicted_new = [max(i,e) for i in y_predicted]
        y_predicted_new = [min(i, 1-e) for i in y_predicted_new]
        y_predicted_new = np.array(y_predicted_new)

        return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

    def _sigmoid_numpy(self, x):
        return 1/(1+np.exp(-x))

In [80]:
my_model = MyNN()
my_model.fit(X_train,y_train, epochs=5000)

Epoch:0 w1:0.9730748899032969, w2:0.948550858114684 bias:-0.11304867552697416 Loss:0.7155451250332395
Epoch:1 w1:0.95192861306694, w2:0.9063298061133309 bias:-0.2114472607052261 Loss:0.6854981016060818
Epoch:2 w1:0.9360637411657075, w2:0.8726924515607353 bias:-0.2965122288261744 Loss:0.663442477143959
Epoch:3 w1:0.9248879811937369, w2:0.8467633240826499 bias:-0.36977288256685903 Loss:0.6475320494611068
Epoch:4 w1:0.9177791255238335, w2:0.8275601298361096 bias:-0.4328127332811064 Loss:0.6361668133242573
Epoch:5 w1:0.9141335852075363, w2:0.8140925114779377 bias:-0.487153442586814 Loss:0.6280606980435903
Epoch:6 w1:0.9133965903980349, w2:0.8054276875148642 bias:-0.5341844083147863 Loss:0.6222367581192162
Epoch:7 w1:0.915077032896223, w2:0.8007264662886235 bias:-0.5751297534241774 Loss:0.6179834225660595
Epoch:8 w1:0.9187515142169604, w2:0.7992574661601929 bias:-0.6110411439309239 Loss:0.6147982160779754
Epoch:9 w1:0.9240618036951638, w2:0.8003974055511585 bias:-0.6428060756295486 Loss:0.6

In [81]:
my_model.get_weights()

(4.865716434849393, 1.466899501833181, -2.878853507272944)

In [82]:
model.get_weights()

[array([[4.873889 ],
        [1.4196932]], dtype=float32),
 array([-2.834229], dtype=float32)]

In [85]:
model.predict(X_test).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([0.3688309 , 0.7883222 , 0.194536  , 0.80413157, 0.82614326,
       0.3688309 ], dtype=float32)

In [84]:
my_model.predict(X_test)

10    0.369089
5     0.787989
13    0.187278
17    0.803791
9     0.825798
19    0.369089
dtype: float64